In [2]:
# CONECTAR CON DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys
path ='/content/drive/MyDrive/cod/LEA3_Marketing'
os.chdir(path) ## volver la carpeta de repositorio directorio de trabajo
sys.path.append(path) ## agregarla al path, para leer archivos propios como paquetes

In [ ]:
#!pip install mlxtend

In [2]:
# LIBRERIAS
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import a_funciones as fn
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql ### paquete para crear y trabajar bases de datos ligeras

In [3]:
# CREAR CONEXIÓN CON LA BASE DE DATOS db_movies
con = sql.connect('data/db_movies')

# CREAR EL CURSOR
cur = con.cursor() ## se crea el cursor, que es el otro tipo de conexión para ejecutar las consultas

In [4]:
# VERIFICAR LOS NOMBRES DE TODAS LAS TABLAS QUE HAY EN LA BASE DE DATOS
cur.execute(""" select name from sqlite_master where type= 'table'  """)
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_selectos',),
 ('pelis_selectas',),
 ('ratings_final',),
 ('movies_final',),
 ('full_rating',),
 ('full_ratings',)]

Se confirma la información del trabajo, y es que la empresa cuenta con una base de datos sql “bd_movies” en la cuál se encuentran **dos tablas**. Una tabla tiene la información del catálogo de películas disponibles en la plataforma llamada ‘movies’. Los campos que tiene esta tabla son:

*  **movieId**: código que identifica la película
*  **title**: Nombre y año de la película
*  **genres**: Lista de géneros a los que pertenece la película.  

La segunda tabla es una lista de los usuarios y las películas que vieron, las fechas en las que las vieron y la calificación que le dieron a la película. Los campos son:

* **userId**: Código que identifica al usuario.
* **movieId**: Código que identifica la película.
* **Rating**: Calificación de la película vista de 1 a 5.
* **Timestamp**: Timestamp de la fecha en la que fue vista la película.


# **TABLAS QUE CONTIENE LA BASE DE DATOS**

In [43]:
# VERIFICAMOS LA PRIMERA TABLA "raitings"
db_ratings = pd.read_sql('SELECT * FROM ratings', con)
db_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
# VERIFICAMOS LA SEGUNDA TABLA "movies"
db_movies = pd.read_sql('SELECT * FROM movies', con)
db_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
db_movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [8]:
db_movies.duplicated().sum()

0

## **Exploración inicial**

In [9]:
# Número de usuarios en la tabla ratings que han calificado peliculas
pd.read_sql("""SELECT COUNT(DISTINCT userId) AS total_usuarios FROM ratings""", con)

,total_usuarios
0,610


Se halla que hay un total de 610 usuarios dentro de la base de datos

In [10]:
#Numero de peliculas en la tabla movies
pd.read_sql("""SELECT COUNT(DISTINCT movieId) AS total_peliculas FROM movies""", con)

,total_peliculas
0,9742


En total se tienen 9742 peliculas dentro de la base de datos

In [11]:
db_ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

Las peliculas van de un rango de 0-5 en intervalos de 0.5

In [12]:
#Conteo de calificaciones
cr = pd.read_sql("""
    SELECT
        rating,
        COUNT(*) AS conteo
    FROM ratings
    GROUP BY rating
    ORDER BY rating ASC
""", con)

# Gráfico de barras
data = go.Bar(
    x=cr.rating,
    y=cr.conteo,
    text=cr.conteo,
    textposition="outside"
)

layout = go.Layout(
    title="Distribución de calificaciones de películas",
    xaxis={
        'title': 'Calificación',
        'type': 'category',  # Tratar cada valor como categoría
        'categoryorder': 'array',
        'categoryarray': cr.rating.tolist()  # Ordenar según tus datos
    },
    yaxis={'title': 'Cantidad'}
)

go.Figure(data=[data], layout=layout).show()

In [5]:
import numpy as np
import plotly.graph_objects as go

# Obtener número de calificaciones por usuario
rating_users = pd.read_sql('''
    SELECT userId AS user_id,
           COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY userId
''', con)

# Definir los rangos (bins más grandes: de 50 en 50)
bin_size = 100  # Tamaño del bin
max_val = rating_users['cnt_rat'].max()
bins = np.arange(0, max_val + bin_size, bin_size)
labels = [f"{b}-{b+bin_size-1}" for b in bins[:-1]]

# Agrupar en los bins
rating_users['bin'] = pd.cut(rating_users['cnt_rat'], bins=bins, labels=labels, right=False)

# Contar cuántos usuarios hay en cada rango
hist_data = rating_users['bin'].value_counts().sort_index()

# Crear gráfico de barras
data = go.Bar(
    x=hist_data.index.astype(str),
    y=hist_data.values,
    text=hist_data.values,
    textposition="outside",
    marker=dict(color='royalblue')
)

layout = go.Layout(
    title="Distribución del número de calificaciones por usuario",
    xaxis={'title': 'Rango de número de calificaciones'},
    yaxis={'title': 'Cantidad de usuarios'}
)

go.Figure(data=[data], layout=layout).show()

Hay demasiados usuarios con un número de calificaciones bajo, por lo tanto es necesario filtrar las calificaciones por usuario. Se idenfica que de los 610 usuarios hay 526 usuarios en las 3 primeras barras, es deicr, tienen 299 calificaciones o menos 

In [16]:
rating_users.describe()

,user_id,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


Se visualiza con boxplot la distribución de las calificaciones de los usuarios

In [18]:
import plotly.graph_objects as go

# Boxplot con Plotly
fig = go.Figure()

fig.add_trace(go.Box(
    x=rating_users['cnt_rat'],
    name='',
    boxpoints='outliers',  # muestra los outliers
    marker=dict(color='skyblue'),
    line=dict(color='darkblue')
))

fig.update_layout(
    title="Número de calificaciones por usuario",
    xaxis_title="Calificaciones",
    yaxis_title="Distribución",
    showlegend=False
)

fig.show()

Del boxplot, se puede observar que el usuario que menos calificaciones ha realizado tiene 20 calificaciones, y se logra identificar que luego de 366 calificaciones (como se observa en el bigote superior) los datos se consideran atipicos, es decir, la mayoria de usarios tiene pocas películas calificadas, pero los que más tienen, son demasiadas, siendo un número improbable.

Por lo anterior y en base a ambos graficos se decide excluir usuarios con más de 300 calificaciones (para obtener calificación confiable) 

In [34]:
# Filtrar usuarios con entre 5 y 500 calificaciones
rating_users2 = pd.read_sql('''
    SELECT userId AS user_id,
           COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY userId
    HAVING cnt_rat < 300
    ORDER BY cnt_rat ASC
''', con)


In [35]:
len(rating_users2)

526

In [36]:

# Crear boxplot filtrado
fig = go.Figure()

fig.add_trace(go.Box(
    x=rating_users2['cnt_rat'],
    name='',
    boxpoints='outliers',  # muestra los outliers
    marker=dict(color='lightcoral'),
    line=dict(color='firebrick')
))

fig.update_layout(
    title="Número de calificaciones por usuario (filtrado)",
    xaxis_title="Número de calificaciones",
    yaxis_title="Distribución",
    showlegend=False
)

fig.show()

In [37]:
# Crear histograma con Plotly
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=rating_users2['cnt_rat'],
    nbinsx=20,  # Número de bins como en matplotlib
    marker=dict(color='skyblue', line=dict(color='black', width=1))
))

fig.update_layout(
    title="Número de calificaciones por usuario (filtrado)",
    xaxis_title="Número de calificaciones",
    yaxis_title="Frecuencia",
    bargap=0.1  # espacio entre barras
)

fig.show()

In [38]:
# Descripción después de los filtros
rating_users2.describe()

,user_id,cnt_rat
count,526.000000,526.000000
mean,303.961977,84.671103
std,173.478755,68.256409
min,1.000000,20.000000
25%,154.250000,33.000000
50%,305.000000,57.000000
75%,451.750000,118.000000
max,609.000000,299.000000


In [40]:
# Verificar cuántas calificaciones tiene cada película
rating_movies = pd.read_sql('''
    SELECT movieId,
           COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY movieId
    ORDER BY cnt_rat DESC
''', con)


In [53]:
# Crear histograma de calificaciones por película
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=rating_movies['cnt_rat'],
    nbinsx=80,  # Igual que matplotlib
    marker=dict(color='steelblue', line=dict(color='black', width=1))
))

fig.update_layout(
    title="Número de calificaciones por película",
    xaxis_title="Número de calificaciones",
    yaxis_title="Frecuencia",
    bargap=0.1
)

fig.show()

Las peliculas que mas calificaciones tienen es por que probablemente son o las taquilleras o las mas antiguas

In [44]:
pd.read_sql('''
    SELECT 
        m.title,
        r.movieId,
        COUNT(r.rating) AS cnt_rat
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY r.movieId, m.title
    ORDER BY cnt_rat desc
    LIMIT 5
''', con)

,title,movieId,cnt_rat
0,Forrest Gump (1994),356,329
1,"Shawshank Redemption, The (1994)",318,317
2,Pulp Fiction (1994),296,307
3,"Silence of the Lambs, The (1991)",593,279
4,"Matrix, The (1999)",2571,278


Efectivamente se comprueba que algunas de estas peliculas son de las conocidas por las personas a nivel general coo Forrest Gump y Matrix

In [48]:
pd.read_sql('''
    SELECT 
        m.title,
        r.movieId,
        COUNT(r.rating) AS cnt_rat
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY r.movieId, m.title
    ORDER BY cnt_rat asc
    LIMIT 5
''', con)

,title,movieId,cnt_rat
0,When Night Is Falling (1995),49,1
1,Georgia (1995),55,1
2,Nico Icon (1995),77,1
3,Once Upon a Time... When We Were Colored (1995),83,1
4,In the Bleak Midwinter (1995),96,1


In [51]:
pd.read_sql('''
    SELECT m.*, r.rating
    FROM movies m
    LEFT JOIN ratings r ON m.movieId = r.movieId
    WHERE r.movieId IS NULL
''', con)

,movieId,title,genres,rating
0,1076,"Innocents, The (1961)",Drama|Horror|Thriller,None
1,2939,Niagara (1953),Drama|Thriller,None
2,3338,For All Mankind (1989),Documentary,None
3,3456,"Color of Paradise, The (Rang-e khoda) (1999)",Drama,None
4,4194,I Know Where I'm Going! (1945),Drama|Romance|War,None
5,5721,"Chosen, The (1981)",Drama,None
6,6668,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance,None
7,6849,Scrooge (1970),Drama|Fantasy|Musical,None
8,7020,Proof (1991),Comedy|Drama|Romance,None
9,7792,"Parallax View, The (1974)",Thriller,None


Cuando se ordena las peliculas de menor a mayor rating, las primeras 5 peliculas que estan en la tabla ratings tienen al menos una calificación, es decir no hay peliculas con 0 calificaciones en esta tabla. No obstante, se confirma que hay peliculas pertenencientes a la tabla movies que no tienen una calificación en rating.

In [45]:
# Distribución de calificaciones por película
rating_movies.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


Se considera que una película con menos de 10 calificaciones no es lo suficientemente confiable para ser recomendada a otros usuarios. Al analizar la distribución de la cantidad de calificaciones por película, se observa que el 75% de las películas tienen menos de 10 calificaciones.

En consecuencia, solo el 25% de las películas (es decir, aquellas en el cuartil superior) cuentan con 10 o más calificaciones y, por tanto, podrían considerarse aptas para generar recomendaciones basadas en opiniones más representativas del público, permitiendo centrarse en títulos más populares y con mayor consenso, lo cual es clave para mejorar la precisión del sistema de recomendación.

In [56]:
####Excluir películas que no tengan más de 10 calificaciones
rating_movies2=pd.read_sql('''
SELECT movieId ,
        COUNT(*) as cnt_rat
FROM ratings
GROUP BY "movieId"
HAVING cnt_rat>9
ORDER BY cnt_rat DESC
''',con )

In [61]:
# Crear histograma de calificaciones por película (filtrado)
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=rating_movies2['cnt_rat'],
    nbinsx=50,  # Igual que en matplotlib
    marker=dict(color='mediumseagreen', line=dict(color='black', width=1))
))

fig.update_layout(
    title="Películas con más de 10 calificaciones",
    xaxis_title="Número de calificaciones",
    yaxis_title="Frecuencia",
    bargap=0.1
)

fig.show()

In [62]:
rating_movies2.describe().T

,count,mean,std,min,25%,50%,75%,max
movieId,2269.0,20530.586161,35185.840333,1.0,1345.0,3256.0,8958.0,187593.0
cnt_rat,2269.0,35.749669,35.986989,10.0,14.0,22.0,43.0,329.0


Efectivamente, el filtro disminuyo las peliculas en aras de recomendar 

In [46]:
fn.ejecutar_sql('preprocesamientos.sql', cur)

cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_selectos',),
 ('pelis_selectas',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [ ]:
con.close() ## Cerrar conexion